In [54]:
import os
import pickle
import datetime as dt
import pandas as pd
from tqdm.notebook import tqdm
import json

# Enable Intellisense for code assisatance and autocomplete
%config IPCompleter.greedy=True

#Install standard modules
import pandas as pd
import numpy as np

# Time modules
import datetime as dt
import datetime
from datetime import timedelta, date, datetime
from dateutil.relativedelta import relativedelta
import time
import calendar


# Yahoo scrapper module
from yahoo_earnings_calendar import YahooEarningsCalendar

# Timezone modules
import pytz

# Progress bar modules
from tqdm.notebook import tqdm

# working days and public holidays module
import pandas_market_calendars as mcal

# API modules
import requests

# json convertor
from pandas.io.json import json_normalize

# Plot Modules
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import seaborn as sns
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')
import os
from fbprophet import Prophet

In [ ]:
# Input AMeritrade API key
api_key = open("Ameritrade_API_key.txt", "r").read()

In [55]:
def get_json_features(pc,temp_df):
    df = pd.DataFrame()
    for c in range(len(pc.loc[0])):
        # don't append quoteTime, only quoteTimeLong
        list_for_dict = ['putCall','strikePrice','mark','theoreticalOptionValue','timeValue',
                         'quoteTimeInLong','volatility','delta','gamma','theta','vega','rho',
                         'openInterest','daysToExpiration','expirationDate','inTheMoney']
        outer_dict_list = ['interestRate', 'underlyingPrice']
        dict_to_append = {x: pc.loc[0][c][0][x] for x in list_for_dict}
        outer_dict = {x: temp_df[x] for x in outer_dict_list}
        dict_to_append.update(outer_dict)
        df = df.append(dict_to_append, ignore_index=True)

    return df

def set_types(df):
    columns_to_float = ['theoreticalOptionValue','volatility','volatility','delta','gamma','theta','rho']
    df[columns_to_float] = df[columns_to_float].astype('float')
    df[['openInterest','daysToExpiration','inTheMoney']] = df[['openInterest','daysToExpiration','inTheMoney']].astype('int32')
    #df['expirationDate'] = pd.to_datetime(df['expirationDate'],unit='ms')
    df['quoteTimeInLong_date'] = pd.to_datetime(df['quoteTimeInLong'],unit='ms')
    #df['quoteTime'] = pd.to_datetime(df['quoteTime'],unit='ms')
    #df['quoteTimeInLong'] = df['quoteTimeInLong'].dt.floor('s')
    #df = df.drop(columns=['putCall','inTheMoney'])
    df['quoteTime'] = df['quoteTimeInLong_date'].astype('datetime64[s]')
    df.quoteTime = pd.to_datetime(df.quoteTime, format ='%y-%m-%d %H:%M:%S')  
    df['quote_date'] = df['quoteTime'].dt.date
    
    return df


def normalize_json_put_call(file_location):
    with open(file_location) as file: 
        temp_df = json.load(file)
        call = pd.json_normalize(temp_df['callExpDateMap'])
        put = pd.json_normalize(temp_df['putExpDateMap'])
        
        # Calls
        df_call = get_json_features(call,temp_df)
        df_call = set_types(df_call)
        #Puts
        df_put = get_json_features(put,temp_df)
        df_put = set_types(df_put)

    return df_call, df_put


def prepare_calendar(df_call_front, df_call_back, df_call_front_exp, df_call_back_exp):

    '''Calculates the realised profit of a calendar spread and cleans the data frame for predicitons

       Enetr 4 dataframes:
        df_call_front: df of the front expirey
        df_call_back: df of the back expirey 
        df_call_front_exp: df of the results for the front at expirey
        df_call_back_exp: df of the results for the back at expirey
    '''

    # select columns to keep
    cols = ['delta', 'gamma', 'inTheMoney', 'mark', 'openInterest','rho', 'strikePrice', 
            'theoreticalOptionValue', 'theta', 'timeValue', 'underlyingPrice', 'vega', 'volatility']
    df_call_front = df_call_front[cols]
    df_call_back = df_call_back[cols]

    # merge for calendar calculations
    merge_call = pd.merge(df_call_front,df_call_back,how='inner',on='strikePrice')
    merge_call = merge_call.dropna()

    # create dataframe for calendar
    calendar_spread = pd.DataFrame()

    # calculate the cost (front - back) different to other calcs
    calendar_spread['mark'] = merge_call['mark_y']-merge_call['mark_x']

    #add the strike
    calendar_spread['strikePrice'] = merge_call['strikePrice']
    calendar_spread['underlyingPrice'] = merge_call['underlyingPrice_x']
    calendar_spread['theoreticalOptionValue'] = merge_call['theoreticalOptionValue_x']-merge_call['theoreticalOptionValue_y']
    calendar_spread['timeValue'] = merge_call['timeValue_x']-merge_call['timeValue_y']

    # difference in greeks (back - front) psoitive of above front
    calendar_spread['volatility'] = merge_call['volatility_y']-merge_call['volatility_x']
    calendar_spread['vega'] = merge_call['vega_y']-merge_call['vega_x']
    calendar_spread['delta'] = merge_call['delta_y']-merge_call['delta_x']
    calendar_spread['gamma'] = merge_call['gamma_y']-merge_call['gamma_x']
    calendar_spread['theta'] = merge_call['theta_y']-merge_call['theta_x']
    calendar_spread['vega'] = merge_call['vega_y']-merge_call['vega_x']
    calendar_spread['rho'] = merge_call['rho_y']-merge_call['rho_x']

    # select columns to keep
    cols = ['delta', 'gamma', 'inTheMoney', 'mark', 'openInterest','rho', 'strikePrice', 
            'theoreticalOptionValue', 'theta', 'timeValue', 'underlyingPrice', 'vega', 'volatility']
    df_call_front_exp = df_call_front_exp[cols]
    df_call_back_exp = df_call_back_exp[cols]

    # merge for calendar calculations
    merge_call_exp = pd.merge(df_call_front_exp,df_call_back_exp,how='inner',on='strikePrice')
    merge_call_exp = merge_call_exp.dropna()

    # create dataframe for calendar
    calendar_spread_exp = pd.DataFrame()

    # calculate the cost (front - back) different to other calcs
    calendar_spread_exp['mark'] = merge_call_exp['mark_y']-merge_call_exp['mark_x']

    #add the strike
    calendar_spread_exp['strikePrice'] = merge_call_exp['strikePrice']

    # merge for results and calculate target variable
    calendar = pd.merge(calendar_spread,calendar_spread_exp,how='inner',on='strikePrice')
    calendar = calendar.dropna()
    calendar['p_L'] = round(calendar['mark_x'] - calendar['mark_y'],2)

    # columns to keep
    cols = ['mark_x', 'strikePrice', 'underlyingPrice', 'theoreticalOptionValue',
           'timeValue', 'volatility', 'vega', 'delta', 'gamma', 'theta', 'rho','p_L']
    calendar = calendar[cols]

    return calendar


In [56]:
def generate_information_from_stock(stk,date_list):
    """Supply one stock"""
    
    print('Fetching ',stk)
    
    df_call_front = pd.DataFrame()
    df_call_back= pd.DataFrame()
    df_call_front_exp= pd.DataFrame()
    df_call_back_exp= pd.DataFrame()
    
    df_put_front = pd.DataFrame()
    df_put_back= pd.DataFrame()
    df_put_front_exp= pd.DataFrame()
    df_put_back_exp= pd.DataFrame()


    df = pd.DataFrame(columns=['underlyingPrice','quoteTime','putCall','strikePrice','mark',
                               'theoreticalOptionValue','timeValue',
                                'quoteTimeInLong','volatility','delta','gamma','theta','vega','rho',
                                'openInterest','daysToExpiration','expirationDate','inTheMoney'])
    
    
    if os.path.exists('option_chain_data/'+stk+'/'+str(ls[0])):
        
        open_date = os.listdir('option_chain_data/'+stk+'/'+str(ls[0]))
        close_date = os.listdir('option_chain_data/'+stk+'/'+str(ls[1]))

        # Data is delayed by 20 minutes
        # 1 hour into the trading day
        open_day = sorted(list(open_date))[7]
        # 1 hour before markets close
        close_day = sorted(list(close_date))[-30]

        with open('option_chain_data/'+stk+'/'+str(ls[0])+'/'+str(open_day)) as file: 
            temp_df = json.load(file)
        #print('is Delayed:', temp_df['isDelayed'])
        if temp_df['status'] == 'SUCCESS':
            open_date_file = 'option_chain_data/'+stk+'/'+str(ls[0])+'/'+str(open_day)
            close_date_file = 'option_chain_data/'+stk+'/'+str(ls[1])+'/'+str(close_day)
            df_call, df_put = normalize_json_put_call(open_date_file)
            df_call_exp, df_put_exp = normalize_json_put_call(close_date_file)
            
            ## CALLS
            ## Start of trade
            # front expirey timestamp
            expirey_front = sorted(df_call.expirationDate.value_counts().index)[0]
            df_call_front = df_call[df_call.expirationDate == expirey_front]
            # back expirey timestamp
            expirey_back = sorted(df_call.expirationDate.value_counts().index)[1]
            df_call_back = df_call[df_call.expirationDate == expirey_back]

            ## End of trade
            # front expirey timestamp
            expirey_front_exp = sorted(df_call_exp.expirationDate.value_counts().index)[0]
            df_call_front_exp = df_call_exp[df_call_exp.expirationDate == expirey_front_exp]
            # back expirey timestamp
            expirey_back_exp = sorted(df_call_exp.expirationDate.value_counts().index)[1]
            df_call_back_exp = df_call_exp[df_call_exp.expirationDate == expirey_back]  
            
            ## PUTS
            ## Start of trade
            # front expirey timestamp
            expirey_front = sorted(df_put.expirationDate.value_counts().index)[0]
            df_put_front = df_put[df_put.expirationDate == expirey_front]
            # back expirey timestamp
            expirey_back = sorted(df_put.expirationDate.value_counts().index)[1]
            df_put_back = df_put[df_put.expirationDate == expirey_back]

            ## End of trade
            # front expirey timestamp
            expirey_front_exp = sorted(df_put_exp.expirationDate.value_counts().index)[0]
            df_put_front_exp = df_put_exp[df_put_exp.expirationDate == expirey_front_exp]
            # back expirey timestamp
            expirey_back_exp = sorted(df_put_exp.expirationDate.value_counts().index)[1]
            df_put_back_exp = df_put_exp[df_put_exp.expirationDate == expirey_back]    
        else:
            print('****  Status = failed *****')

    return df_call_front, df_call_back, df_call_front_exp, df_call_back_exp, df_put_front, df_put_back, df_put_front_exp, df_put_back_exp 


In [57]:
# Find available stocks in data bank
stock_list = os.listdir('option_chain_data/')
if '.DS_Store' in stock_list:
    stock_list.remove('.DS_Store')


In [ ]:
to_concat = pd.DataFrame()
total_call_df = pd. DataFrame()
total_put_df = pd. DataFrame()
i=0
j=0
ls = ['2020-11-23','2020-11-27']    

for stk in tqdm(stock_list):
    a,b,c,d,e,f,g,h = generate_information_from_stock(stk,ls)

    if (len(a) == 0) or (len(b) == 0) or (len(c) == 0) or (len(d) == 0):
        print("****Calls no data ****")
        pass
    else:
        to_concat = prepare_calendar(a,b,c,d)
        to_concat['stock'] = i
        to_concat['stock_scale'] = stk
        total_call_df = pd.concat([total_call_df,to_concat])
        i+=1
        
    if (len(e) == 0) or (len(f) == 0) or (len(g) == 0) or (len(h) == 0):
        print("****Calls no data ****")
        pass
    else:
        to_concat = prepare_calendar(e,f,g,h)
        to_concat['stock'] = j
        to_concat['stock_scale'] = stk
        total_put_df = pd.concat([total_put_df,to_concat])
        j+=1
        
total_call_df = total_call_df.reset_index(drop=True)
total_put_df = total_put_df.reset_index(drop=True)

In [ ]:
total_call_df = total_call_df.reset_index(drop=True)
total_put_df = total_put_df.reset_index(drop=True)

In [ ]:
# Export to CSV
total_call_df.to_csv('total_call_df_week_47.csv',index=False)
total_put_df.to_csv('total_put_df_week_47.csv',index=False)

In [ ]:
df_classifier_call = pd.read_csv('total_call_df_week_47.csv')

# Generate Classification scheme
df_classifier_call['p_L'] = df_classifier_call['p_L'] * -1
df_classifier_call['Percent_profit'] = (df_classifier_call['p_L']/df_classifier_call['mark_x'])*100
df_classifier_call['classifier'] = df_classifier_call['Percent_profit'].apply(lambda x: 1 if x >= 50 else 0)
df_classifier_call['p_L_10'] = df_classifier_call['p_L'].apply(lambda x: 1 if x >= 0.20 else 0)
df_classifier_call['total_score'] = df_classifier_call['p_L_10'] + df_classifier_call['classifier']
df_classifier_call['classifier'] = df_classifier_call['total_score'].apply(lambda x: 1 if x == 2  else 0)
df_classifier_call = df_classifier_call.drop(columns=['p_L_10','total_score'])

# export csv
df_classifier_call.to_csv('df_classifier_call.csv',index=False)
df_classifier_call.classifier.value_counts()

In [ ]:
df_classifier_put = pd.read_csv('total_put_df_week_47.csv')

# Generate Classification scheme
df_classifier_put['p_L'] = df_classifier_put['p_L'] * -1
df_classifier_put['Percent_profit'] = (df_classifier_put['p_L']/df_classifier_put['mark_x'])*100
df_classifier_put['classifier'] = df_classifier_put['Percent_profit'].apply(lambda x: 1 if x >= 50 else 0)
df_classifier_put['p_L_10'] = df_classifier_put['p_L'].apply(lambda x: 1 if x >= 0.20 else 0)
df_classifier_put['total_score'] = df_classifier_put['p_L_10'] + df_classifier_put['classifier']
df_classifier_put['classifier'] = df_classifier_put['total_score'].apply(lambda x: 1 if x == 2  else 0)
df_classifier_put = df_classifier_put.drop(columns=['p_L_10','total_score'])

# export csv
df_classifier_put.to_csv('df_classifier_put.csv',index=False)
df_classifier_put.classifier.value_counts()

In [11]:
def historical_data_FB_profit(ticker,ls):
    

     # define our endpoint
    endpoint = r"https://api.tdameritrade.com/v1/marketdata/{}/pricehistory".format(ticker)

     # define our payload
    payload = {'apikey':api_key,
               'periodType': 'year',
               'period' : '5',
               'frequencyType':'daily'
              }

     # make a request
    content = requests.get(url = endpoint, params = payload)

    # flatten data and create Dataframe
    data = content.json()
    data = pd.json_normalize(data['candles'])
    # convert timestamp to just a date
    data['date_time'] = pd.to_datetime(data['datetime'],unit='ms')
    data['date'] = data['date_time'].dt.date
    data['percent_move'] = ((data['close']-data['open'])/data['open'])*100
   
    # change this later
    mask = data['date_time'] <= '2020-11-24'
    data = data.loc[mask]
    data = data.reset_index(drop=True)
    
    # Prepare for FB profit
    data = data[["date","close"]]
    data = data.rename(columns = {"date":"ds","close":"y"})
    
    data['y'] = np.log(data['y'] )

    m = Prophet(daily_seasonality = True)
    m.fit(data) 
    future = m.make_future_dataframe(periods=7,include_history=True) #we need to specify the number of days in future
    prediction = m.predict(future)
    prediction['yhat'] = np.exp(prediction['yhat'])
    prediction['yhat_upper'] = np.exp(prediction['yhat_upper'])
    prediction['yhat_lower'] = np.exp(prediction['yhat_lower'])
    prediction = prediction[prediction.ds == ls[1]]

    time.sleep(0.5)
    
    exp_date_pred = prediction[['yhat','yhat_upper','yhat_lower']]

    return  exp_date_pred

In [12]:
phrophet_var_call = pd.read_csv('df_classifier_call.csv')
phrophet_var_put = pd.read_csv('df_classifier_put.csv')

In [59]:
ls = ['2020-11-23','2020-11-27']    

prophet_yhat = {}
prophet_yhat_upper = {}
prophet_yhat_lower = {}

for i in tqdm(stock_list): 
    try:
        prophet_pred = historical_data_FB_profit(i,ls)
        prophet_yhat.update({i:prophet_pred.iloc[0][0]})
        prophet_yhat_upper.update({i:prophet_pred.iloc[0][1]})
        prophet_yhat_lower.update({i:prophet_pred.iloc[0][2]})
    except:
        pass
        print(i, "fails")


2020-12-01 21:35:11,701 fbprophet    INFO     Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.


AET fails
PCLN fails



In [47]:
# Call
phrophet_var_call['prophet_yhat'] = phrophet_var_call['stock_scale'].map(prophet_yhat)
phrophet_var_call['prophet_yhat_upper'] = phrophet_var_call['stock_scale'].map(prophet_yhat_upper)
phrophet_var_call['prophet_yhat_lower'] = phrophet_var_call['stock_scale'].map(prophet_yhat_lower)
# Put
phrophet_var_put['prophet_yhat'] = phrophet_var_put['stock_scale'].map(prophet_yhat)
phrophet_var_put['prophet_yhat_upper'] = phrophet_var_put['stock_scale'].map(prophet_yhat_upper)
phrophet_var_put['prophet_yhat_lower'] = phrophet_var_put['stock_scale'].map(prophet_yhat_lower)

In [50]:
# Export to CSV
phrophet_var_call.to_csv('phrophet_var_call.csv',index=False)
phrophet_var_put.to_csv('phrophet_var_put.csv',index=False)